#Импорты

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Activation
import zipfile
import os

In [3]:
!python --version

Python 3.10.12


# Скачивание и генерация данных

In [ ]:
zip_file_path = '/content/handwritten.zip'
output_folder = '/content/data'

os.makedirs(output_folder, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_folder)

print(f'Файлы успешно распакованы в папку: {output_folder}')

Файлы успешно распакованы в папку: /content/data


In [ ]:
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import random

# Путь к файлу шрифта
font_path = '/content/ocr-b.ttf'
font_size = 24

output_folder = '/content/data/printed'
os.makedirs(output_folder, exist_ok=True)

font = ImageFont.truetype(font_path, font_size)

with open('/content/words.txt', 'r', encoding='utf-8') as file:
    words = file.readlines()

# Функция для добавления шума
def add_noise(image):
    img_array = np.array(image)
    noise = np.random.normal(0, 25, img_array.shape)
    noisy_image = img_array + noise

    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

    return Image.fromarray(noisy_image)

for word in words:
    word = word.strip()
    if word:
        image = Image.new('RGB', (100, 50), (255, 182, 193))
        draw = ImageDraw.Draw(image)
        draw.text((10, 10), word, font=font, fill=(0, 0, 0))

        rnd = random.randint(0, 5)
        if rnd < 3:
            rotated_image = image.rotate(180)
        else:
            rotated_image = image

        noisy_image = add_noise(rotated_image)
        output_path = os.path.join(output_folder, f"{word}.png")
        noisy_image.save(output_path)

print("Изображения успешно созданы и сохранены в папку:", output_folder)

Изображения успешно созданы и сохранены в папку: /content/data/printed


# Train датасет

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    '/content/data',
    target_size=(150, 150),
    batch_size=50,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/data',
    target_size=(150, 150),
    batch_size=50,
    class_mode='binary',
    subset='validation'
)

Found 2540 images belonging to 2 classes.
Found 635 images belonging to 2 classes.


# Модель

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(150, 150, 3)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_generator,
          validation_data=validation_generator,
          epochs=10)

Epoch 1/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 16s 182ms/step - accuracy: 0.9302 - loss: 0.1511 - val_accuracy: 0.9323 - val_loss: 0.3432
Epoch 2/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.9963 - loss: 0.0190 - val_accuracy: 0.8488 - val_loss: 0.3031
Epoch 3/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.9877 - loss: 0.0351 - val_accuracy: 0.9323 - val_loss: 0.2557
Epoch 4/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 96ms/step - accuracy: 0.9947 - loss: 0.0142 - val_accuracy: 0.8457 - val_loss: 0.2933
Epoch 5/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.9959 - loss: 0.0138 - val_accuracy: 0.5150 - val_loss: 0.8226


In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.5254 - loss: 0.8180
Validation Accuracy: 51.50%


In [ ]:
model.save('my_model.keras')

# Тестирование

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
img_path = '/content/ex22.png'  # Замените на фактический путь к изображению

# Загружаем изображение и изменяем его размер
img = image.load_img(img_path, target_size=(150, 150))

# Преобразуем изображение в массив
img_array = image.img_to_array(img)

# Добавляем дополнительное измерение (для батча)
img_array = np.expand_dims(img_array, axis=0)

# Нормализуем изображение
img_array /= 255.0  # Если вы использовали rescale=1./255 при обучении

In [ ]:
# Получаем предсказание
predictions = model.predict(img_array)

# Преобразуем вероятности в класс
predicted_class = (predictions > 0.8).astype("int32")  # Для бинарной классификации

# Выводим результат
print(f'Предсказанный класс: {predicted_class[0][0]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Предсказанный класс: 0
